# Geospatial Analysis for the Smart City

Big Data BBQ 9/24/2021

* https://github.com/gboeing/osmnx
* https://osmnx.readthedocs.io/en/stable/


In [ ]:
import folium
import osmnx as ox
import networkx as nx
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
ox.config(use_cache=True, log_console=False)

### Retrieve Street Network from OSM Data

* drive - get drivable public streets (but not service roads)
* drive_service - get drivable streets, including service roads
* walk - get all streets and paths that pedestrians can use (this network type ignores one-way directionality)
* bike - get all streets and paths that cyclists can use
* all - download all non-private OSM streets and paths
* all_private - download all OSM streets and paths, including private-access ones

In [ ]:
place = "Mannheim, Germany"
graph = ox.graph_from_place(place, network_type='drive')

In [ ]:
fig, ax = ox.plot_graph(graph)

#### Export als ESRI Shapefile

In [ ]:
ox.save_graph_shapefile(graph, filepath='network-shape_mannheim')

In [ ]:
nodes, streets = ox.graph_to_gdfs(graph)

In [ ]:
len(streets)

In [ ]:
streets.head()

In [ ]:
street_types = pd.DataFrame(streets["highway"].apply(pd.Series)[0].value_counts().reset_index())
street_types.columns = ["type", "count"]

In [ ]:
street_types

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
sns.barplot(y="type", x="count", data=street_types, ax=ax)
plt.tight_layout()
plt.show();

### Display Street Network on a Map

In [ ]:
m = folium.Map([49.473, 8.475],zoom_start=13,tiles="CartoDB dark_matter")

jsondata = streets.to_json()

style = {'color': '#FFDD66', 
         'weight':'1'}

folium.GeoJson(jsondata, style_function=lambda x: style).add_to(m)
m

### Routing


In [ ]:
# Train Station
pos_1 = ox.geocode('Mannheim HBF, Mannheim, Germany')
pos_1

In [ ]:
# MAFINEX
pos_2 = ox.geocode('Julius-Hatry-Straße 1, 68163 Mannheim, Germany') 
pos_2

#### Display on Map

In [ ]:
m = folium.Map([49.473, 8.475], zoom_start=13,tiles="CartoDB dark_matter")

#jsondata = streets.to_json()

style = {'color': '#FFDD66', 
         'weight':'1'}
folium.GeoJson(jsondata, style_function=lambda x: style).add_to(m)

folium.Marker(pos_1, 
              popup="Mannheim HBF",
              icon=folium.Icon(color="green", prefix="fa", icon="train")).add_to(m)

folium.Marker(pos_2, 
              popup="MAFINEX",
              icon=folium.Icon(color="red", prefix="fa", icon="star")).add_to(m)
m

In [ ]:
origin_node = ox.get_nearest_node(graph, pos_1)
destination_node = ox.get_nearest_node(graph, pos_2)

#### Route finden

In [ ]:
route = nx.shortest_path(graph, origin_node, destination_node)

In [ ]:
fig, ax = ox.plot_graph_route(graph, route)

In [ ]:
from shapely.geometry import LineString

line = []
for i in route:
    point = (graph.nodes[i]['x'],graph.nodes[i]['y'])
    line.append(point)
    print(point)
    
ls = LineString(line)

#### GeoDataFrame from LineString


In [ ]:
gdf_route = pd.DataFrame(data= [['route 1',ls]],columns=['route','geometry'])
gdf_route

In [ ]:
gdf_route = gpd.GeoDataFrame(gdf_route,geometry='geometry')
gdf_route

In [ ]:
gdf_route.plot();

Convert this route to GeoJSON

In [ ]:
jsonroute = gdf_route.to_json()

In [ ]:
m = folium.Map([49.473, 8.475], zoom_start=13,tiles="CartoDB dark_matter")

#jsondata = streets.to_json()

style = {'color': '#FFDD66', 
         'weight':'1'}

route_style = {'color': '#FF0000',
              'weight' : '4'}

folium.GeoJson(jsondata, style_function=lambda x: style).add_to(m)
folium.GeoJson(jsonroute, style_function=lambda x: route_style).add_to(m)

folium.Marker(pos_1, 
              popup="Mannheim HBF",
              icon=folium.Icon(color="green", prefix="fa", icon="train")).add_to(m)

folium.Marker(pos_2, 
              popup="MAFINEX",
              icon=folium.Icon(color="red", prefix="fa", icon="star")).add_to(m)
m

### Building Footprints

In [ ]:
buildings = ox.geometries_from_place("Mannheim, Germany", tags={'building':True})

buildings.shape

In [ ]:
buildings.head()

In [ ]:
ox.plot_footprints(buildings, figsize=(16,15));

### Retrieve some data

In [ ]:
museum = buildings.query("tourism == 'museum'")
museum = museum[['name', 'geometry']]
museum.head(20)

In [ ]:
ox.plot_footprints(museum, figsize=(16,15));

### Darstellung auf Karte

In [ ]:
import html

m = folium.Map([49.473, 8.475],zoom_start=15)


style = {'color': '#FF0000',
         'fillColor': '#FFFF00',
         'weight':'2'
        }

def building(building):
    jsondata = gpd.GeoSeries([building["geometry"]]).to_json()
    name = html.escape(building["name"]) 
    folium.GeoJson(jsondata, style_function=lambda x: style).add_child(folium.Popup(name)).add_to(m)

museum.apply(building, axis=1)

m.save("museums.html")
m

### Marker from Buildings

In [ ]:
def get_geometry(x):
    g = x["geometry"]
    c = g.centroid
    return c
   
centroid = museum.apply(get_geometry, axis=1)

In [ ]:
museum["centroid"] = centroid 

In [ ]:
museum.head()

In [ ]:
import html

m = folium.Map([49.473, 8.475],zoom_start=15)


def building(building):
    lng = building["centroid"].x
    lat = building["centroid"].y

    name = html.escape(building["name"])
   
    folium.Marker([lat,lng], 
              popup=name,
              icon=folium.Icon(color="red", prefix="fa", icon="university")).add_to(m)


museum.apply(building, axis=1)

m

### Get Points of interest from OSM

In [ ]:
museum = ox.pois_from_place("Mannheim, Germany", tags={'tourism': 'museum'})

In [ ]:
len(museum)

In [ ]:
museum.head()

In [ ]:
import html
import shapely

m = folium.Map([49.473, 8.475],zoom_start=15)


def mus2poi(x):
    if type(x["geometry"]) == shapely.geometry.point.Point:
        lng = x["geometry"].x
        lat = x["geometry"].y


        folium.Marker([lat,lng], 
                  popup=x["name"],
                  icon=folium.Icon(color="green", prefix="fa", icon="university")).add_to(m)


museum.apply(mus2poi, axis=1)

m